# Investment Analysis

In this notebook, you will learn how to use crewai to create agents that collaborate to provide investment analysis.

If you're running this notebook on your own machine, you can install the following:

!pip install crewai==0.83.0 crewai-tools==0.14.0 langchain-community==0.3.8 duckduckgo_search==6.3.6  

## Create a conda environment
 
Run: 
conda create -n crewai python=3.12
conda activate crewai

Next install the following modules:
pip install crewai==0.83.0 crewai-tools==0.14.0 langchain-community==0.3.8 duckduckgo_search==6.3.6 ipykernel PyPDF2==3.0.1


**References:**
- https://github.com/ksm26/Multi-AI-Agent-Systems-with-crewAI

# crewAI flow

```mermaid
graph TD
    Crew[Investment Analysis Crew] -->|uses| Agent1[Summarizer]
    Crew[Investment Analysis Crew] -->|uses| Agent2[Market Analyst]
    Crew[Investment Analysis Crew] -->|uses| Agent3[Financial Analyst]
    Crew[Investment Analysis Crew] -->|uses| Agent4[Manager]

    Agent1[Summarizer] -->|performs| Task1[Read PDF Task]
    Agent2[Market Analyst] -->|performs| Task2[Market Analysis Task]
    Agent3[Financial Analyst] -->|performs| Task3[Financial Analysis Task]
    Agent4[Manager] -->|performs| Task4[Report Creation Task]

    Task1[Read PDF Task] -->|uses| Tool1[Pdf Reader Tool]
    Task2[Market Analysis Task] -->|uses| Tool2[Scrape Website Tool]
    Task2[Market Analysis Task] -->|uses| Tool3[DuckDuckGo Search Results Tool]
    Task3[Financial Analysis Task] -->|uses| Tool2[Scrape Website Tool]
    Task3[Financial Analysis Task] -->|uses| Tool3[DuckDuckGo Search Results Tool]
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Import libraries, APIs and LLM

In [ ]:
from crewai import Agent, Task

# Configure Azure OpenAI variables

In [ ]:
import os

os.environ["AZURE_OPENAI_VERSION"] = "2023-03-15-preview"
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "gpt-4o"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://fabricaio.openai.azure.com"
# Read the environment variable. 
azure_openai_key = os.getenv("AZURE_API_KEY")


print("Loaded environment variables for crewai successfully!")

# Define the default crewai LLM

In [ ]:
from langchain_openai import AzureChatOpenAI

default_llm = AzureChatOpenAI(
    openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://fabricaio.openai.azure.com/"),
    api_key=azure_openai_key
)

# crewAI tools

In [ ]:
from crewai_tools import ScrapeWebsiteTool
from langchain_community.tools import DuckDuckGoSearchResults
scrape_tool = ScrapeWebsiteTool()


# crewAI custom tool

For being able to read a pdf file the following custom tool needs to be created that extracts text from a pdf file.

This tools is being created using crewAI's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [ ]:
from crewai_tools import BaseTool
from PyPDF2 import PdfReader
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper


- Every Tool needs to have a name and a description.
- When running locally, you can customize the code with your logic in the _run function.

In [ ]:
# https://pypi.org/project/duckduckgo-search/#regions
# https://python.langchain.com/api_reference/community/utilities/langchain_community.utilities.duckduckgo_search.DuckDuckGoSearchAPIWrapper.html#langchain_community.utilities.duckduckgo_search.DuckDuckGoSearchAPIWrapper.region
# https://python.langchain.com/docs/integrations/tools/ddg/

class CustomDuckDuckGoSearchResults(BaseTool):
    name: str = "DuckDuckGo Search Results Tool"
    description: str = "Generates a web search query based on the input text."

    def _run(self, query: str) -> str:
        """
        Generates a web search query based on the input text.

        Args:
            query (str): The input text to generate the search query.

        Returns:
            str: The web search query generated from the input text.
        """
        wrapper = DuckDuckGoSearchAPIWrapper(region="nl-nl", max_results=5)
        #search = DuckDuckGoSearchResults(output_format="json")
        search = DuckDuckGoSearchResults(api_wrapper=wrapper, output_format="json")
        return search.invoke(query)

In [ ]:
search_tool = CustomDuckDuckGoSearchResults()

In [ ]:
class PdfReaderTool(BaseTool):
    name: str ="Pdf Reader Tool"
    description: str = ("Reads the content of a PDF file and returns the text.")
    
    def _run(self, file_path: str) -> str:
        """
        Reads the content of a PDF file and returns the text.

        Args:
            file_path (str): The path to the PDF file.

        Returns:
            str: The text content of the PDF file.
        """
        with open(file_path, "rb") as file:
            reader = PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
        return text

In [ ]:
pdf_reader_tool = PdfReaderTool()

# crewAI agents

In [ ]:
summarizer_agent = Agent(
    role="Summarizer",
    goal="Summarize the content of a PDF file into a concise report.",
    backstory="A summarizer agent reads the content of a PDF file and summarizes it into a concise report."
              "They identify key points, extract important information, and provide a summary of the document."
              "Always respond in the {language} language provided.",
    verbose=True,
    allow_delegation=False,
    tools=[pdf_reader_tool]
)

In [ ]:
market_analyst_agent = Agent(
    role="Market Analyst",
    goal="Analyze market trends and provide insights to help investors make informed decisions."
         "Always respond in the {language} language provided.",
    backstory="A market analyst agent examines the market conditions and competitive landscape"
              "in which the company operates."
              "They assess market trends, demand, competition, and potential growth opportunities."
              "Their analysis helps determine the market potential and"
              "strategic positioning of the company.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [ ]:
financial_analyst_agent = Agent(
    role="Financial Analyst",
    goal="Analyze financial health and performance {industry} and provide insights to help investors make informed decisions."
         "Always respond in the {language} language provided.",
    backstory="A financial analyst evaluates the financial health and performance of the company seeking investment."
                "They analyze financial statements, assess profitability, liquidity, and solvency,"
                "and perform financial modeling to project future performance."
                "Their goal is to determine the financial viability of the investment.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
    
)

In [ ]:
manager_agent = Agent(
    role="Manager",
    goal="Write the final investment analysis report based on the input from the other agents"
        "Always respond in the {language} language provided."
        "The report needs to be at least 6000 characters long!",
    backstory="You are the most senior investment manager of the company and responsible for the end report being sent out to the client."
                "Your goal is to review the content from the other agents"
                "and write the final report being sent out to the clientrequesting this report."
                "Make sure it contains all the relevant information so that the client can make solid decision on wether to invest in this company or not.",
    verbose=True,
    allow_delegation=False
)

# crewAI Tasks

In [ ]:
# Create a crewAI task for reading the pdf file with an argument of the file path

class PdfReaderTask:
    def read_pdf_task(self, file_path):
        return Task(
        description=(
            "Summarize the key points from the provided PDF document located at {file_path},"
        ),
        expected_output="A concise summary of the key points from the PDF document.",
        agent=summarizer_agent,
        tools=[pdf_reader_tool],
        output_file="outputs/summary.md",
)
        
pdf_reader_task = PdfReaderTask().read_pdf_task(
            'C://Temp//InformatieMemorandumStatuta.pdf',
        )

In [ ]:
# Task for Market Analyst Agent: Analyze Market Data
market_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected industry ({industry} and {country}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for industry ({industry} and {country})."
        "All your finding need to be supported by the search urls you have used during your analysis!"
    ),
    agent=market_analyst_agent,
    output_file="outputs/market_analysis.md",
)

In [ ]:
# Task for Financial Analyst Agent: Analyze financial Data
financial_analysis_task = Task(
    description=(
        "Continuously monitor and analyze financial data for "
        "the selected industry ({industry} and {country}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict financial results."
    ),
    expected_output=(
        "Insights and alerts about significant financial "
        "opportunities or threats for industry ({industry} and {country})."
        "All your finding need to be supported by the search urls you have used during your analysis!"
    ),
    agent=financial_analyst_agent,
    output_file="outputs/financial_analysis.md",
)

In [ ]:
# Final report creation task
report_creation_task = Task(
    description=(
        "Final investment analysis report for the client"
        "The following sections need to be part of the report"
        "1. Summary: Summary of the provide input pdf file"
        "2. Strengths Identification: Identify the strong points of the proposed investment request."
        "3. Weaknesses Identification: Highlight potential weaknesses and risks."
        "4. Financial Insights: Describe the company's financial statements, and include financial insights from online research."
        "5. Marktet Insights: Describe the market for which an investment is asked, describe the industry, market trends, and competitive landscape."
        "6. Final Conclusion: Describe if the client should make the investment or not, with a clear explaination of the reasons."
    ),
    expected_output=(
        "A well written report in markdown format, ready to be sent out the client."
    ),
    output_file="outputs/investment_analysis_report.md",
    agent=manager_agent,
)

# crewAI crew

- The Process class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- manager_llm lets you choose the "manager" LLM you want to use.

In [ ]:
from crewai import Crew , Process

# Define the crew with agents and tasks
investment_crew = Crew(
    agents=[
        summarizer_agent,
        market_analyst_agent,
        financial_analyst_agent,
        manager_agent
        ],    
    tasks=[
        pdf_reader_task,
        market_analysis_task,
        financial_analysis_task,
        report_creation_task
        ],    
    #manager_llm=default_llm,     
    process='sequential',
    verbose=True,
    memory=True
)

# Running the crew

- Set the inputs for the execution of the crew

In [ ]:
# Example data for kicking off the process
investment_inputs = {
    'industry': 'AI software voor rechtspraak',
    'country': 'Nederland',
    'language': 'Dutch',
    'search-region': 'nl-nl',
    'file_path': 'C://Temp//InformatieMemorandumStatuta.pdf'
}

In [ ]:
### this execution will take some time to run
result = investment_crew.kickoff(inputs=investment_inputs)